In [10]:
import sympy as sp

# --- symbols ---
m, l, k, p = sp.symbols('m l k p', integer=True)
x, y, z     = sp.symbols('x y z', real=True)

# --- g(m,l,k) ---
g = lambda m,l,k: (
    (-1)**k
    * 2**(-l)
    * sp.binomial(l, k)
    * sp.binomial(2*l - 2*k, l)
    * sp.factorial(l - 2*k)
    / sp.factorial(l - 2*k - m)
)

# --- P(m,l) ---
P = lambda m,l: sp.summation(
    g(m,l,k)
    * (x**2 + y**2 + z**2)**k
    * z**(l - 2*k - m),
    (k, 0, sp.floor((l - m)/2))
)

# --- A(m) and B(m) ---
A = lambda m: sp.summation(
    sp.binomial(m, p)
    * x**p
    * y**(m - p)
    * sp.cos((m - p)*sp.pi/2),
    (p, 0, m)
)
B = lambda m: sp.summation(
    sp.binomial(m, p)
    * x**p
    * y**(m - p)
    * sp.sin((m - p)*sp.pi/2),
    (p, 0, m)
)

# --- Rp(m,l) and Rm(m,l) ---
Rp = lambda m,l: sp.sqrt(
    (2 - sp.KroneckerDelta(m, 0))
    * sp.factorial(l - m)
    / sp.factorial(l + m)
) * P(m, l) * A(m)

Rm = lambda m,l: sp.sqrt(
    2*sp.factorial(l - m)
    / sp.factorial(l + m)
) * P(m, l) * B(m)

In [22]:
# --- choose a specific l ---
l_val = 7

cpp_code = []
for m_val in range(-l_val, l_val+1):
    # pick expression and name
    if m_val < 0:
        expr   = Rm(abs(m_val), l_val)
        name   = f"Y_real_l{l_val}m{m_val}"
    else:
        expr   = Rp(m_val, l_val)
        name   = f"Y_real_l{l_val}m{m_val}"
    # simplify and ccode it
    expr_s = sp.simplify(expr)
    body   = sp.ccode(expr_s, assign_to=None)
    
    # emit a static inline C++ function
#     cpp_code.append(f"""\
# static inline double {name}(double x, double y, double z) {{
#     return {body};
# }}
# """)
    cpp_code.append(f"""buffer[{l_val + m_val}] = {body};""")


# join and print
print("\n".join(cpp_code))

buffer[0] = sqrt(429)*y*(0.21875*pow(x, 6) - 1.09375*pow(x, 4)*pow(y, 2) + 0.65625*pow(x, 2)*pow(y, 4) - 0.03125*pow(y, 6));
buffer[1] = sqrt(6006)*x*y*z*(0.1875*pow(x, 4) - 0.625*pow(x, 2)*pow(y, 2) + 0.1875*pow(y, 4));
buffer[2] = -1.0/166320.0*sqrt(231)*y*(5197.5*pow(x, 2) + 5197.5*pow(y, 2) - 62370.0*pow(z, 2))*(5*pow(x, 4) - 10*pow(x, 2)*pow(y, 2) + pow(y, 4));
buffer[3] = (1.0/6930.0)*sqrt(231)*x*y*z*(pow(x, 2) - pow(y, 2))*(-5197.5*pow(x, 2) - 5197.5*pow(y, 2) + 17325.0*pow(z, 2));
buffer[4] = (1.0/1260.0)*sqrt(21)*y*(3*pow(x, 2) - pow(y, 2))*(5630.625*pow(z, 4) - 2598.75*pow(z, 2)*(pow(x, 2) + pow(y, 2) + pow(z, 2)) + 118.125*pow(pow(x, 2) + pow(y, 2) + pow(z, 2), 2));
buffer[5] = (1.0/126.0)*sqrt(42)*x*y*z*(1126.125*pow(z, 4) - 866.25*pow(z, 2)*(pow(x, 2) + pow(y, 2) + pow(z, 2)) + 118.125*pow(pow(x, 2) + pow(y, 2) + pow(z, 2), 2));
buffer[6] = (1.0/14.0)*sqrt(7)*y*(187.6875*pow(z, 6) - 216.5625*pow(z, 4)*(pow(x, 2) + pow(y, 2) + pow(z, 2)) + 59.0625*pow(z, 2)*pow(pow(x, 2) + 

In [13]:
import sympy as sp

# --- symbols ---
m, k = sp.symbols('m k', integer=True)
x, y, z = sp.symbols('x y z', real=True)

# --- Mathematica‐style building blocks ---
g = lambda m,l,k: (
    (-1)**k
    * 2**(-l)
    * sp.binomial(l, k)
    * sp.binomial(2*l - 2*k, l)
    * sp.factorial(l - 2*k)
    / sp.factorial(l - 2*k - m)
)
P = lambda m,l: sp.summation(
    g(m,l,k) * (x**2 + y**2 + z**2)**k * z**(l-2*k-m),
    (k, 0, sp.floor((l-m)/2))
)
A = lambda m: sp.summation(
    sp.binomial(m, p)*x**p*y**(m-p)*sp.cos((m-p)*sp.pi/2),
    (p, 0, m)
)
B = lambda m: sp.summation(
    sp.binomial(m, p)*x**p*y**(m-p)*sp.sin((m-p)*sp.pi/2),
    (p, 0, m)
)
Rp = lambda m,l: sp.sqrt((2 - sp.KroneckerDelta(m,0))
                         * sp.factorial(l-m)
                         / sp.factorial(l+m)
                        ) * P(m,l) * A(m)
Rm = lambda m,l: sp.sqrt(2*sp.factorial(l-m)/sp.factorial(l+m)) * P(m,l) * B(m)


def generate_cpp_for_l(l_val):
    # 1) precompute numeric prefactors for m=1..l  and m=0..l 
    #    as pure double literals
    pref_Rm = [float(sp.N(sp.sqrt(2*sp.factorial(l_val-m)/sp.factorial(l_val+m))))
               for m in range(1, l_val+1)]
    pref_Rp = [float(sp.N(sp.sqrt((2-sp.KroneckerDelta(m,0))
                                 *sp.factorial(l_val-m)
                                 /sp.factorial(l_val+m))))
               for m in range(0, l_val+1)]

    lines = []
    lines.append(f"// --- real spherical harmonics (l = {l_val}) ---")
    lines.append(f"static const double pref_Rm_l{l_val}[{l_val}] = {{ {', '.join(f'{v:.12g}' for v in pref_Rm)} }};")
    lines.append(f"static const double pref_Rp_l{l_val}[{l_val+1}] = {{ {', '.join(f'{v:.12g}' for v in pref_Rp)} }};")
    lines.append("")
    lines.append(f"static inline void eval_X_l{l_val}(double x, double y, double z, double *out) {{")
    lines.append("    double r2 = x*x + y*y + z*z;")
    # negative‐m (Rm) in decreasing m = l..1
    for idx, m_val in enumerate(range(l_val, 0, -1)):
        expr = sp.simplify(Rm(m_val, l_val) / pref_Rm[idx])
        cexpr = sp.ccode(expr, assign_to=None)
        lines.append(f"    // out[{idx}] = Rm(m={m_val})")
        lines.append(f"    out[{idx}] = pref_Rm_l{l_val}[{idx}] * ({cexpr});")
    # non-negative m (Rp) in m = 0..l
    base = l_val
    for m_val in range(0, l_val+1):
        expr = sp.simplify(Rp(m_val, l_val) / pref_Rp[m_val])
        cexpr = sp.ccode(expr, assign_to=None)
        idx = base + m_val
        lines.append(f"    // out[{idx}] = Rp(m={m_val})")
        lines.append(f"    out[{idx}] = pref_Rp_l{l_val}[{m_val}] * ({cexpr});")
    lines.append("}")
    return "\n".join(lines)


# example: emit for l=2
print(generate_cpp_for_l(2))

// --- real spherical harmonics (l = 2) ---
static const double pref_Rm_l2[2] = { 0.57735026919, 0.288675134595 };
static const double pref_Rp_l2[3] = { 1, 0.57735026919, 0.288675134595 };

static inline void eval_X_l2(double x, double y, double z, double *out) {
    double r2 = x*x + y*y + z*z;
    // out[0] = Rm(m=2)
    out[0] = pref_Rm_l2[0] * (1.7320508075688774*sqrt(3)*x*y);
    // out[1] = Rm(m=1)
    out[1] = pref_Rm_l2[1] * (3.4641016151377548*sqrt(3)*y*z);
    // out[2] = Rp(m=0)
    out[2] = pref_Rp_l2[0] * (-0.5*pow(x, 2) - 0.5*pow(y, 2) + 1.0*pow(z, 2));
    // out[3] = Rp(m=1)
    out[3] = pref_Rp_l2[1] * (1.7320508075688774*sqrt(3)*x*z);
    // out[4] = Rp(m=2)
    out[4] = pref_Rp_l2[2] * (1.7320508075688774*sqrt(3)*(pow(x, 2) - pow(y, 2)));
}
